### Dynamic programming 時間複雜度O(n^3)

In [1]:
import re
from pprint import pprint

In [2]:
#此段程式負責處理字串分離，將運算元跟運算子分離並判斷正負號


def input_math_expression():
    math_expression = input("請輸入運算式 (ex:3+6*8，限於'+'、'-'、'*')：")
    expression_operand = list(map(int,re.findall(r'(\d{1,100})',math_expression))) #找出所有數字並轉換成int矩陣(正負號未處理)
    expression_operator = []

    record = ['']                         #處理字串的紀錄，第一個預設''
    record.extend(list(math_expression))
    record_index = 1
    sign_index = 0
    if len(math_expression)>30:
        print("不合法輸入")
    else:
        for i in math_expression:
            if(i=='-' or i=='+'or i=='*'):           #判斷'-'是運算子還是負運算元，如果是'-'判斷前一個字元是否是數字
                if (not record[record_index-1].isdigit() and i=='-'):
                    expression_operand[sign_index]*=-1
                else:
                    expression_operator.append(i)
                    sign_index +=1
            record_index+=1
    print("運算元:", expression_operand)
    print("運算子:", expression_operator)
    
    return (expression_operand,expression_operator)

expression = input_math_expression()

請輸入運算式 (ex:3+6*8，限於'+'、'-'、'*')：5-8+7*4-8+9
運算元: [5, 8, 7, 4, 8, 9]
運算子: ['-', '+', '*', '-', '+']


In [3]:

expression_operand = expression[0]
expression_operator = expression[1]
s_max = [[[None,0,None]]*len(expression_operand) for i in range(len(expression_operand))]
s_min = [[[None,0,None]]*len(expression_operand) for i in range(len(expression_operand))]

def parenthesize_expression():
    
    #生成紀錄最大值矩陣跟最小值矩陣
    MEMOIZED_MATRIX_Max = [[float("-inf")]*len(expression_operand) for i in range(len(expression_operand))]
    for i in range(0,len(expression_operand)):
        MEMOIZED_MATRIX_Max[i][i]=float(expression_operand[i])
    MEMOIZED_MATRIX_Min = [[float("inf")]*len(expression_operand) for i in range(len(expression_operand))]
    for i in range(0,len(expression_operand)):
        MEMOIZED_MATRIX_Min[i][i]=float(expression_operand[i])

    for l in range(1,len(expression_operand)): #1~3
        for i in range(0,len(expression_operand)-l):#0~4-l
            j=i+l
            for k in range(i,j):
                temp=[]
                temp.append(Calculation(MEMOIZED_MATRIX_Max[i][k],MEMOIZED_MATRIX_Min[k+1][j],k))
                temp.append(Calculation(MEMOIZED_MATRIX_Min[i][k],MEMOIZED_MATRIX_Max[k+1][j],k))
                temp.append(Calculation(MEMOIZED_MATRIX_Max[i][k],MEMOIZED_MATRIX_Max[k+1][j],k))
                temp.append(Calculation(MEMOIZED_MATRIX_Min[i][k],MEMOIZED_MATRIX_Min[k+1][j],k))
                if max(temp)>MEMOIZED_MATRIX_Max[i][j]:
                    MEMOIZED_MATRIX_Max[i][j]=max(temp)
                    s_max[i][j] = record(temp.index(max(temp)),k)
                if min(temp)<MEMOIZED_MATRIX_Min[i][j]:
                    MEMOIZED_MATRIX_Min[i][j]=min(temp)
                    s_min[i][j] = record(temp.index(min(temp)),k)
    pprint(MEMOIZED_MATRIX_Max)
    print()
    pprint(MEMOIZED_MATRIX_Min)
    
def record(index,k):
    if (index==0):
        return ["s_max",k,"s_min"]
    elif (index==1):
        return ["s_min",k,"s_max"]
    elif (index==2):
        return ["s_max",k,"s_max"]
    elif (index==3):
        return ["s_min",k,"s_min"]
    
def Calculation(index_1, index_2, index):
    if(expression_operator[index]=='+'):
        return index_1+index_2
    elif(expression_operator[index]=='*'):
        return index_1*index_2
    elif(expression_operator[index]=='-'):
        return index_1-index_2

parenthesize_expression()

[[5.0, -3.0, 4.0, 25.0, 65.0, 200.0],
 [-inf, 8.0, 15.0, 60.0, 52.0, 75.0],
 [-inf, -inf, 7.0, 28.0, 20.0, 35.0],
 [-inf, -inf, -inf, 4.0, -4.0, 5.0],
 [-inf, -inf, -inf, -inf, 8.0, 17.0],
 [-inf, -inf, -inf, -inf, -inf, 9.0]]

[[5.0, -3.0, -10.0, -55.0, -63.0, -94.0],
 [inf, 8.0, 15.0, 36.0, -60.0, -195.0],
 [inf, inf, 7.0, 28.0, -28.0, -91.0],
 [inf, inf, inf, 4.0, -4.0, -13.0],
 [inf, inf, inf, inf, 8.0, 17.0],
 [inf, inf, inf, inf, inf, 9.0]]


In [4]:
def parentheses(left, right, max_or_min): #搜尋i~j

    if (max_or_min=="s_max"):#判斷用用哪個矩陣追蹤
        temp=s_max
    else:
        temp=s_min
    
    if (left!=right):
        print("(",end="")
        parentheses(left, temp[left][right][1], temp[left][right][0]) #往左
        print(expression_operator[temp[left][right][1]],end="")
        parentheses(temp[left][right][1]+1, right, temp[left][right][2]) #往右
        print(")",end="")
    else:
        print(expression_operand[right],end="")

In [5]:
print("最大值：", end="")
parentheses(0,len(expression_operand)-1,"s_max") #最大

print("\n最小值：", end="")
parentheses(0,len(expression_operand)-1,"s_min") #最小

最大值：(5-((8+7)*(4-(8+9))))
最小值：((5-8)+(7*(4-(8+9))))